In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/cud

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 

In [7]:
import seaborn as sns
pourboire = sns.load_dataset("tips")
pourboire.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [13]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
df = spark.createDataFrame(pourboire)
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: long (nullable = true)



/content/spark-2.3.1-bin-hadoop2.7/python/pyspark/sql/session.py:665: UserWarning: Arrow will not be used in createDataFrame: Unsupported type in conversion from Arrow: dictionary<values=string, indices=int8, ordered=0>
  warnings.warn("Arrow will not be used in createDataFrame: %s" % str(e))


In [16]:
df2 = df.withColumnRenamed("total_bill","Total") \
    .withColumnRenamed("tip","pourboire") \
    .withColumnRenamed("sex","genre") \
    .withColumnRenamed("smoker","fumeur") \
    .withColumnRenamed("day","jour") \
    .withColumnRenamed("time","repas") \
    .withColumnRenamed("size","nombre") \

In [18]:
df2.show(3)

+-----+---------+------+------+----+------+------+
|Total|pourboire| genre|fumeur|jour| repas|nombre|
+-----+---------+------+------+----+------+------+
|16.99|     1.01|Female|    No| Sun|Dinner|     2|
|10.34|     1.66|  Male|    No| Sun|Dinner|     3|
|21.01|      3.5|  Male|    No| Sun|Dinner|     3|
+-----+---------+------+------+----+------+------+
only showing top 3 rows



In [22]:
df2.count()

244

In [24]:
df2.filter(df2['pourboire'] > 2).filter(df2['nombre'] == 2).show(10)

+-----+---------+------+------+----+------+------+
|Total|pourboire| genre|fumeur|jour| repas|nombre|
+-----+---------+------+------+----+------+------+
|23.68|     3.31|  Male|    No| Sun|Dinner|     2|
|14.78|     3.23|  Male|    No| Sun|Dinner|     2|
|14.83|     3.02|Female|    No| Sun|Dinner|     2|
|21.58|     3.92|  Male|    No| Sun|Dinner|     2|
|17.92|     4.08|  Male|    No| Sat|Dinner|     2|
|20.29|     2.75|Female|    No| Sat|Dinner|     2|
|15.77|     2.23|Female|    No| Sat|Dinner|     2|
|19.82|     3.18|  Male|    No| Sat|Dinner|     2|
| 21.7|      4.3|  Male|    No| Sat|Dinner|     2|
|19.65|      3.0|Female|    No| Sat|Dinner|     2|
+-----+---------+------+------+----+------+------+
only showing top 10 rows



In [25]:
df2.filter(df2['pourboire'] > 2).filter(df2['nombre'] == 2).filter(df2['fumeur'] == 'Yes').show(10)

+-----+---------+------+------+----+------+------+
|Total|pourboire| genre|fumeur|jour| repas|nombre|
+-----+---------+------+------+----+------+------+
|20.29|     3.21|  Male|   Yes| Sat|Dinner|     2|
|15.01|     2.09|  Male|   Yes| Sat|Dinner|     2|
|26.86|     3.14|Female|   Yes| Sat|Dinner|     2|
|25.28|      5.0|Female|   Yes| Sat|Dinner|     2|
|17.92|     3.08|  Male|   Yes| Sat|Dinner|     2|
|19.44|      3.0|  Male|   Yes|Thur| Lunch|     2|
|32.68|      5.0|  Male|   Yes|Thur| Lunch|     2|
|28.97|      3.0|  Male|   Yes| Fri|Dinner|     2|
|16.32|      4.3|Female|   Yes| Fri|Dinner|     2|
|27.28|      4.0|  Male|   Yes| Fri|Dinner|     2|
+-----+---------+------+------+----+------+------+
only showing top 10 rows



In [26]:
df2.filter(df2['pourboire'] > 2).filter(df2['nombre'] == 2).filter(df2['fumeur'] == 'Yes').count()

41

In [28]:
df2.filter(df2['pourboire'] > 2).filter(df2['nombre'] == 2).filter(df2['fumeur'] == 'Yes').orderBy('pourboire',ascending=False).show(10)

+-----+---------+------+------+----+------+------+
|Total|pourboire| genre|fumeur|jour| repas|nombre|
+-----+---------+------+------+----+------+------+
|23.33|     5.65|  Male|   Yes| Sun|Dinner|     2|
| 7.25|     5.15|  Male|   Yes| Sun|Dinner|     2|
|25.28|      5.0|Female|   Yes| Sat|Dinner|     2|
|32.68|      5.0|  Male|   Yes|Thur| Lunch|     2|
|16.32|      4.3|Female|   Yes| Fri|Dinner|     2|
|25.21|     4.29|  Male|   Yes| Sat|Dinner|     2|
|19.81|     4.19|Female|   Yes|Thur| Lunch|     2|
|20.49|     4.06|  Male|   Yes| Sat|Dinner|     2|
|14.31|      4.0|Female|   Yes| Sat|Dinner|     2|
|16.82|      4.0|  Male|   Yes| Sun|Dinner|     2|
+-----+---------+------+------+----+------+------+
only showing top 10 rows



In [29]:
df2.groupBy('genre').count().show()

+------+-----+
| genre|count|
+------+-----+
|Female|   87|
|  Male|  157|
+------+-----+



In [33]:
df2.groupBy('jour').count().orderBy('count',ascending=False).show(10)

+----+-----+
|jour|count|
+----+-----+
| Sat|   87|
| Sun|   76|
|Thur|   62|
| Fri|   19|
+----+-----+

